In [5]:
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("/Users/Masih/Desktop/bootcamp /Challenges/python-api-challenge/WeatherPy./Starter_Code/csv.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,floro,61.5996,5.0328,278.19,87,75,13.89,NO,1705824582
1,1,longyearbyen,78.2186,15.6401,257.06,72,75,3.60,SJ,1705824583
2,2,bilibino,68.0546,166.4372,251.89,95,100,1.37,RU,1705824583
3,3,waitangi,-43.9535,-176.5597,292.49,94,100,1.79,NZ,1705824584
4,4,albany,42.6001,-73.9662,262.97,74,33,2.24,US,1705824584


In [40]:
%%capture --no-display

# Configure the scatter plot
scatter_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    scale=1,  # Adjust the scale value to make the dots smaller
    color="City",
    colormap="viridis",
    title="City Humidity",
    hover_cols=["City", "Humidity"]
)

scatter_plot



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [46]:
ideal_weather_df = city_data_df[(city_data_df['Max Temp'] < 270)  & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]
ideal_weather_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,bethel,41.3712,-73.4140,265.38,70,0,2.06,US,1705824526
104,104,qaqortoq,60.7167,-46.0333,255.64,62,0,3.54,GL,1705824611
147,147,hamilton,39.1834,-84.5333,260.89,79,0,3.58,US,1705824610
227,227,fort bragg,35.1390,-79.0060,267.15,48,0,3.60,US,1705824655
262,262,homer,59.6425,-151.5483,269.08,62,0,2.06,US,1705824682
275,275,badger,64.8000,-147.5333,250.10,70,0,3.60,US,1705824653
327,327,murghob,38.1702,73.9667,261.96,49,0,1.76,TJ,1705824706
358,358,college,64.8569,-147.8028,250.14,64,0,3.60,US,1705824534
396,396,palmer,42.1584,-72.3287,262.94,61,0,4.12,US,1705824731
443,443,westport,41.1415,-73.3579,266.90,66,0,4.12,US,1705824617


In [52]:

# Add an empty column, "Hotel Name," to the DataFrame

hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
hotel_df = hotel_df.assign(Hotel_Name='')

hotel_df.head(30)

,City,Country,Lat,Lng,Humidity,Hotel_Name
5,bethel,US,41.3712,-73.4140,70,
104,qaqortoq,GL,60.7167,-46.0333,62,
147,hamilton,US,39.1834,-84.5333,79,
227,fort bragg,US,35.1390,-79.0060,48,
262,homer,US,59.6425,-151.5483,62,
275,badger,US,64.8000,-147.5333,70,
327,murghob,TJ,38.1702,73.9667,49,
358,college,US,64.8569,-147.8028,64,
396,palmer,US,42.1584,-72.3287,61,
443,westport,US,41.1415,-73.3579,66,


In [55]:
# Set parameters to search for a hotel
radius = 100000
params = {
    "filter": "category_ids:accommodation",
    "bias": "proximity",
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"category_ids:accommodation"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params).json()
    
    # Check if the response contains any hotels
    if "features" in response and len(response["features"]) > 0:
        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        hotel_name = response["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
    else:
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")


Starting hotel search
bethel - nearest hotel: No hotel found
qaqortoq - nearest hotel: No hotel found
hamilton - nearest hotel: No hotel found
fort bragg - nearest hotel: No hotel found
homer - nearest hotel: No hotel found
badger - nearest hotel: No hotel found
murghob - nearest hotel: No hotel found
college - nearest hotel: No hotel found
palmer - nearest hotel: No hotel found
westport - nearest hotel: No hotel found
broad ripple - nearest hotel: No hotel found
kinston - nearest hotel: No hotel found
metropolis - nearest hotel: No hotel found
